# Nanosafety data annotation

In [ ]:
import yaml
from pynanomapper import aa
from pynanomapper import units
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr
from pynanomapper import client_ambit
from pynanomapper import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import importlib
import qgrid
import warnings
import seaborn as sns

import matplotlib.pyplot as plt
import io
import qgrid
from measurement.utils import guess
from measurement.measures import Distance, Temperature, Volume, Area, Frequency, Mass, Time
#init_notebook_mode(connected=True)
warnings.simplefilter("ignore")

import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')
global logger
logger = logging.getLogger()
logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())


In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))
facets = client_solr.Facets()


In [ ]:
try:
    with open('./config.solr.json') as f:
        config = json.load(f)
    solr_query=config["query"]   
except Exception as err:        
    config = None
    logger.error("Something went wrong",err)


In [ ]:
term_widget = widgets.Dropdown(
    options=solr_query.keys(),
    description='term:',
    disabled=False,
)

display(term_widget)

In [ ]:
project=service_widget.value
solr_url=service_uri + "/select"
#nested facet
synonym_facet=",facet: {field2: {"+ "{}:{},{}:{} ,limit : -1, mincount:1 ".format("type","terms","field",solr_query[term_widget.value]["synonym"]) +"}}"
json_facet="{field1: {" + "{}:{},{}:{} ,limit : -1, mincount:1 ".format("type","terms","field",solr_query[term_widget.value]["field"]) + synonym_facet + " }}"

query={'q': solr_query[term_widget.value]["q"],"wt" : "json", "json.facet": json_facet, 'rows': 0}
logger.debug(solr_url)
logger.info(query)

In [ ]:
  
r = requests.post(solr_url,data=query, auth=auth_object ,stream=False)
logger.info(r.status_code)
#print(r.json())

In [ ]:
import pandas as pd
key1 = "field1"
key2 = "field2"

fields = r.json()["facets"][key1]["buckets"]
fields_name=[]
fields_count=[]
field_synonym=[]
for value in fields:
    fields_name.append(value['val'])
    fields_count.append(value['count'])
    _synonym=''
    try:
        for synonym in value[key2]['buckets']:
            if synonym['val'].startswith("EP_"):
                continue
            _synonym = _synonym + " " + synonym['val']

    except Exception as err:
        pass
    field_synonym.append(_synonym.strip())    

df = pd.DataFrame({term_widget.value : fields_name, "count" : fields_count, key2 : field_synonym})
df

In [ ]:
propertiesfile_widget=widgets.Text(
    placeholder='',
    description='Annotation file',
    disabled=False
    
)
display(propertiesfile_widget)


In [ ]:
import os, os.path
from jproperties import Properties

p = Properties()
if (solr_query[term_widget.value]['annotation']!=''):
    if propertiesfile_widget.value == '':
        prop_file='./annotation/'+ solr_query[term_widget.value]['annotation']
    else:
        prop_file= os.path.normcase(os.path.join(propertiesfile_widget.value,solr_query[term_widget.value]['annotation']))
    try:
        with open(prop_file, "rb") as f:
            p.load(f, "utf-8") 

    except Exception as err:
        p = None
        print(err)
    

In [ ]:
def _annotate(x):
    if p is None:
        return None
    x=x.replace(" ","_").upper().strip()
    
    if x in p:
        value, meta = p[x]
        return value
    else:
        return None
    

df["_annotation"] = df[term_widget.value].apply(lambda x: ( _annotate(x)))
df[[term_widget.value,"_annotation"]]

In [ ]:
def store(p):
    if p != None:
        with open('./annotation/' + solr_query[term_widget.value]['annotation'], "wb") as f:
            p.store(f, encoding="utf-8")
#store(p)            